In [ ]:
import sys
sys.path.append('/usr/local/lib/python3/dist-packages/')

In [ ]:
import sys
import os
import lzma
import random
from collections import defaultdict
import math

In [ ]:
from typing import *

In [ ]:
import numpy
import pandas

In [ ]:
import xgboost as xgb

In [ ]:
import sklearn

In [ ]:
from catboost import Pool, CatBoostClassifier
import catboost

In [ ]:
from util import *

In [ ]:
cancer_data_dir = '/home/noskill/projects/cancer/data'
dump_dir = os.path.join(cancer_data_dir, 'bcDump/example15bmc')
clinical_table_path = os.path.join(cancer_data_dir, 'bcClinicalTable.csv')
merged_path = os.path.join(dump_dir, 'ex15bmcMerged.csv.xz')
bmc_all_path = os.path.join(dump_dir, 'bmc15mldata1.csv')

In [ ]:
dtype = {'DFS': pandas.Int64Dtype(),
         'pCR': pandas.Int64Dtype(),
         'RFS': pandas.Int64Dtype(), 
         'DFS': pandas.Int64Dtype(), 
         'posOutcome': pandas.Int64Dtype()}

# load averaged treatment table

In [ ]:
bmc = pandas.read_csv(bmc_all_path, dtype=dtype, converters=converters)
bmc = bmc.sort_values(by='patient_ID')

# load detailed treatment

In [ ]:
treatment = pandas.read_csv(clinical_table_path, converters=converters).sort_values(by='patient_ID')
treatment = treatment[treatment.patient_ID.isin(bmc.patient_ID)]

In [ ]:
bmc.head()

# load genes expression data

In [ ]:
gene_expression = pandas.read_csv(lzma.open(merged_path))

In [ ]:
gene_expression.head(5)

In [ ]:
genes_features = gene_expression[gene_expression.patient_ID.isin(bmc.patient_ID)]

In [ ]:
genes_features = genes_features.sort_values(by='patient_ID')

# columns to use for training

In [ ]:
pam50col = genes_features.columns[genes_features.columns.isin(pam50).nonzero()[0]].to_list()

In [ ]:
aggregated_treatment_columns = ['radio', 'surgery', 'chemo', 'hormone']
label_columns = ['pCR', 'RFS', 'DFS', 'posOutcome']
label_columns = ['posOutcome']
genes_columns = genes_features.columns.to_list()[1:]
feature_columns = xgboost_top_100 #genes_columns + treatment_columns # label_columns +  # pam50col #  +   + aggregated_treatment_columns

## merge genes expression + averaged treatment + detailed treatment

In [ ]:
merged = pandas.merge(genes_features, bmc, left_on='patient_ID', right_on='patient_ID')
merged = pandas.merge(merged, treatment, left_on='patient_ID', right_on='patient_ID')
merged.insert(0, 'row_num', range(0,len(merged)))

# catboost

In [ ]:
res = defaultdict(list)
model = CatBoostClassifier(iterations=3600,
                           depth=7,
                           use_best_model=True,
                           learning_rate=0.005,
                           loss_function='Logloss',
                           model_size_reg=20,
                           verbose=True,
                           scale_pos_weight=0.605,
                           l2_leaf_reg=20,
                           od_type='Iter', od_wait=200)
model = CatBoostClassifier(verbose=True)
train_data, train_labels, val_data, val_labels, expected = random_split(balance_by_study=True)
catboost_pool = Pool(train_data, 
                    train_labels)

test_data = Pool(val_data,
                 val_labels) 
# train the model
clf = model.fit(train_data, train_labels, 
          eval_set=test_data,
          save_snapshot=False, snapshot_file='vasya')
y_pred = clf.predict(val_data)
x_pred = clf.predict(train_data)
compute_metrics(res, val_labels.flatten(), y_pred, train_labels, x_pred)
res

# xgboost

In [ ]:
import xgboost as xgb
res = defaultdict(list)

for i in range(30):
    print(i)
    model = xgb.XGBClassifier()

    train_data, train_labels, val_data, val_labels, expected = random_split(merged, bmc,
                                                                           feature_columns, label_columns)

    # train the model
    clf = model.fit(train_data, train_labels,
                    eval_set=[(train_data, train_labels), (val_data, val_labels)], 
                    early_stopping_rounds=50, verbose=False)
    y_pred = clf.predict(val_data)
    x_pred = clf.predict(train_data)
    compute_metrics(res, val_labels.flatten(), y_pred, train_labels, x_pred)
for key in res:
    ave = numpy.asarray(res[key]).mean(axis=0)
    print('{0}: {1}'.format(key, ave))

In [ ]:
print(train_data.shape)
print(val_data.shape)

In [ ]:
weights = model.get_booster().get_score(importance_type="gain")

In [ ]:
weights_sorted = sorted([(y, x) for (x,y) in weights.items()], reverse=True)

In [ ]:
[feature_columns[int(num[1:])] for (_, num) in weights_sorted[:100]]

In [ ]:
for (w, num) in weights_sorted[:100]:
    print(feature_columns[int(num[1:])], w)

# SVM

In [ ]:
from sklearn import datasets, svm, metrics
svm_total = defaultdict(list)
for i in range(5):
    print('iteration {0}'.format(i))
    model = svm.SVC(C=1, kernel='rbf', class_weight={1: 0.5})
    train_data, train_labels, val_data, val_labels, expected = random_split()
    # train the model
    clf = model.fit(numpy.nan_to_num(train_data), numpy.nan_to_num(train_labels))
    y_pred = clf.predict(numpy.nan_to_num(val_data))
    x_pred = clf.predict(numpy.nan_to_num(train_data))
    compute_metrics(svm_total, val_labels.flatten(), y_pred, train_labels, x_pred)
for key in svm_total:
    ave = numpy.asarray(svm_total[key]).mean(axis=0)
    print('{0}: {1}'.format(key, ave))

In [ ]:
print(train_data.shape)
print(val_data.shape)

# SVM - single study

In [ ]:
from sklearn import datasets, svm, metrics

svm_total = defaultdict(list)
for study in set(bmc.study):
#     if study != 'study_25065_GPL96_MDACC-bmc15':
#         continue
    print('\n' * 2 + study)
    for i in range(20):
        train_data, train_labels, val_data, val_labels, expected = random_split(ratio=0.1, study_name=study)
        C = 1
        if study == 'study_16446_GPL570_all-bmc15':
            C = 1.5
        if study == 'study_22358_GPL5325_all-bmc15':
            C = 0.1
        if study == 'study_22226_GPL1708_all-bmc15':
            C = 2
        if study == 'study_20181_GPL96_all-bmc15':
            C = 0.72
        if study == 'study_25065_GPL96_MDACC-bmc15':
            C = 1.72
        model = svm.SVC(C=C, kernel='rbf', class_weight={1: (1 - numpy.mean(train_labels))  / numpy.mean(train_labels)})
        # train the model
        clf = model.fit(numpy.nan_to_num(train_data), numpy.nan_to_num(train_labels))
        y_pred = clf.predict(numpy.nan_to_num(val_data))
        # print(y_pred)
        x_pred = clf.predict(numpy.nan_to_num(train_data))
        compute_metrics(svm_total, val_labels.flatten(), y_pred, train_labels, x_pred)
for key in svm_total:
    ave = numpy.asarray(svm_total[key]).mean(axis=0)
    print('{0}: {1}'.format(key, ave))

In [ ]:
print(train_data.shape)
print(val_data.shape)

# nearest neigbour classifier

In [ ]:
def predict(train_data, validation_data, train_labels):
    tmp = []
    for i in range(len(validation_data)):
        diff = train_data - validation_data[i]
        idx = numpy.argmin(numpy.sqrt(numpy.sum(diff ** 2, axis=1)))
        tmp.append(idx)
    return train_labels[tmp]

In [ ]:
nearest_total = defaultdict(list)
for i in range(10):
    train_data, train_labels, val_data, val_labels, expected = random_split()
    y_pred = predict(numpy.nan_to_num(train_data), numpy.nan_to_num(val_data), train_labels)
    # x_pred = predict(numpy.nan_to_num(train_data), numpy.nan_to_num(train_data), train_labels)
    compute_metrics(nearest_total, val_labels.flatten(), y_pred, train_labels, x_pred)

for key in nearest_total:
    ave = numpy.asarray(nearest_total[key]).mean(axis=0)
    print('{0}: {1}'.format(key, ave))

In [ ]:
print(train_data.shape)
print(val_data.shape)

# naive bayes

In [ ]:
from sklearn.naive_bayes import GaussianNB
train_data, train_labels, val_data, val_labels, expected = random_split(ratio=0.1)

In [ ]:
nearest_total = defaultdict(list)
for i in range(10):
    gnb = GaussianNB()
    train_data, train_labels, val_data, val_labels, expected = random_split()
    model = gnb.fit(numpy.nan_to_num(train_data, nan=-1), numpy.nan_to_num(train_labels))
    y_pred = model.predict(numpy.nan_to_num(val_data, nan=-1))
    x_pred = model.predict(numpy.nan_to_num(train_data))
    compute_metrics(nearest_total, val_labels.flatten(), y_pred, train_labels, x_pred)

for key in nearest_total:
    ave = numpy.asarray(nearest_total[key]).mean(axis=0)
    print('{0}: {1}'.format(key, ave))



In [ ]:
train_data.shape

# binarization

In [ ]:
merged.head()

In [ ]:
bin_data = binarize_dataset(merged, genes_columns, feature_columns, to_letters=False)

In [ ]:
#bin_data.drop('patient_ID', axis=1, inplace=True)

## test binarization by xgboost

In [ ]:
for col in bin_data.dtypes[bin_data.dtypes == object].keys():
    col_dict = {x: i for (i, x) in enumerate(sorted(set(bin_data[col])))}
    bin_data[col] = bin_data[col].apply(lambda x: col_dict[x])

In [ ]:
import xgboost as xgb
res = defaultdict(list)

for i in range(10):
    print(i)
    model = xgb.XGBClassifier()

    train_data, train_labels, val_data, val_labels, expected = random_split(bin_data, bmc,
                                                                           feature_columns, label_columns)

    # train the model
    clf = model.fit(train_data, train_labels,
                    eval_set=[(train_data, train_labels), (val_data, val_labels)], 
                    early_stopping_rounds=50, verbose=False)
    y_pred = clf.predict(val_data)
    x_pred = clf.predict(train_data)
    compute_metrics(res, val_labels.flatten(), y_pred, train_labels, x_pred)
for key in res:
    ave = numpy.asarray(res[key]).mean(axis=0)
    print('{0}: {1}'.format(key, ave))

In [ ]:
subset_moses_features = ['posOutcome'] + feature_columns

In [ ]:
bin_data[subset_moses_features].to_csv('/tmp/cancer_bin_100.csv', header=True, index=False)

In [ ]:
subset = bin_data[subset_moses_features]

In [ ]:
subset.columns[17]

In [ ]:
bin_data[subset_moses_features[:30]].head(20).to_csv('/tmp/cancer_bin_100_small.csv', header=True, index=False)

In [ ]:
res_ar[0][-4:]

In [ ]:
[12 < 2 ** x for x in range(1,5)]

In [ ]:
for oth in other_columns:
    print(oth)
    print(set(digitize_non_genes_data(merged[oth])))

In [ ]:
other_columns[6]

In [ ]:
set(numpy.nan_to_num(merged.hormone_therapy, nan=-1))

In [ ]:
other_columns[0]

In [ ]:
numpy.histogram_bin_edges(r, bins=14)

In [ ]:
other_columns[18]

In [ ]:
edges = numpy.histogram_bin_edges(r, bins=14)

In [ ]:
edges

# moses

In [ ]:
from opencog.pyasmoses import moses

In [ ]:
train_data, train_labels, val_data, val_labels, expected = random_split(ratio=0.1, to_numpy=False)

In [ ]:
train_data.shape

save data to file

In [ ]:
train_data = train_data.fillna(-1)
train_data.to_csv('/tmp/input_data.csv', header=True, index=False)

In [ ]:
set(train_data.surgery_type.to_list())

In [ ]:
input_data = numpy.concatenate([train_labels[..., numpy.newaxis], train_data], axis=1)

In [ ]:
print(input_data.shape)

In [ ]:
mos = moses()

In [ ]:
args = "--log-file log1.txt.log --hc-fraction-of-nn 0.01 -j5 --balance 1 -m 100000 --result-count 100 --reduct-knob-building-effort=2 --hc-widen-search=1 --enable-fs=1 --fs-algo=smd --fs-target-size=1000 --hc-crossover-min-neighbors=5000 --fs-focus=all --fs-seed=init  --hc-max-nn-evals=10000 --hc-crossover-pop-size=1000 -l debug --noise 0.2 -q 0.05"

In [ ]:
output = mos.run(input=numpy.nan_to_num(input_data), python=True, args=args)

In [ ]:
output[0].score

In [ ]:
mos = moses()
input_data = [[0, 0, 0],
              [0.2, 0.2, 0.4],
              [1, 1, 2],
              [1, 0, 1],
              [2., 1, 3]]
output = mos.run(input=input_data, python=True, args='-m 1000000 --max-time=60 --balance=1')
print (output[0].score) # Prints: 0
model = output[0].eval
print(model([0, 1]))  # Returns: True
print(model([1, 1]))  # Returns: False

In [ ]:
moses_args = []
# target column
moses_args.append('--problem_type=it')
